In [1]:
!pip install flask_ngrok 

In [2]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, url_for, request

In [3]:
import pickle
import pandas as pd
import datetime as dt
import re
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor

In [4]:
# All the Genres Values
flatten_genres_all = [ 12,  21,  25,  26,  27,  31,  36,  45,  53,  58,  64,  66,  70,
        71,  83,  85,  88,  89,  90,  98, 100, 101, 109, 111, 113, 167,
       314, 359, 361, 404, 439, 440, 539, 542, 580, 811]

In [5]:
# One Hot Encoding to Lanuage Codes
df_all = pd.DataFrame({'l':['en', 'tr', 'es', 'id', 'zh', 'sr', 'pt', 'fr', 'nl', 'ru', 'de', 'cs', 'it']})

OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_l = pd.DataFrame(OH_encoder.fit_transform(df_all[['l']]))
OH_l.index=df_all.index
print(OH_l)

     0    1    2    3    4    5    6    7    8    9    10   11   12
0   0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
1   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
2   0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3   0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
5   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
6   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
7   0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
8   0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
9   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
10  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
11  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
12  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0


In [9]:
app = Flask(__name__, template_folder='templates')
run_with_ngrok(app)
@app.route('/',methods=["GET", 'POST'])
def predict():
    if request.method == "POST": 
       # getting input with name = title in HTML form 
       title = request.form.get("title") 
       bit_rate = request.form.get("bit_rate", type =int)
       comments = request.form.get("comments", type =int) 
       duration = request.form.get("duration", type =int)
       favorites = request.form.get("favorites", type =int) 
       interest = request.form.get("interest", type =int)
       listens = request.form.get("listens", type =int) 
       number = request.form.get("number", type =int)

       date_created = request.form.get("date_created")
       date_created = pd.to_datetime(date_created, format="%Y %m %d %H")
       year_created = date_created.year
       date_recorded = request.form.get("date_recorded")
       date_recorded = pd.to_datetime(date_recorded, format="%Y %m %d %H")
       year_recorded = date_recorded.year
       recording_period = abs((date_created - date_recorded).days)

       genres_all = request.form.get("genres_all")
       for i in range(len(flatten_genres_all)):
         globals()['genres_%s' % flatten_genres_all[i]] = 1 if (str(flatten_genres_all[i]) in re.findall('\d+', genres_all)) else 0
      
       acousticness = request.form.get("acousticness", type=float)
       danceability = request.form.get("danceability", type=float)
       energy = request.form.get("energy", type=float) 
       instrumentalness = request.form.get("instrumentalness", type=float)
       liveness = request.form.get("liveness", type=float) 
       speechiness = request.form.get("speechiness", type=float)
       tempo = request.form.get("tempo", type=float) 
       valence = request.form.get("valence", type=float)

       language_code = request.form.get("combo")
       df_lang = pd.DataFrame({'l':[str(language_code)]})
       OH_cols = pd.DataFrame(OH_encoder.transform(df_lang[['l']]))
       #print(OH_cols.loc[0,:12])

       song_info = [bit_rate, comments, duration,
                    favorites, interest, listens,
                    number, year_created, year_recorded,
                    recording_period, genres_25, genres_26,
                    genres_27, genres_31, genres_36,
                    genres_45, genres_53, genres_58,
                    genres_64, genres_66, genres_70,
                    genres_71, genres_83, genres_85,
                    genres_88, genres_89, genres_90,
                    genres_98, genres_100, genres_101,
                    genres_109, genres_111, genres_113,
                    genres_167, genres_314, genres_359,
                    genres_361, genres_404, genres_439,
                    genres_440, genres_539, genres_542,
                    genres_580, genres_811, acousticness,
                    danceability, energy, instrumentalness,
                    liveness, speechiness, tempo,
                    valence]
       song_info.extend(OH_cols.loc[0,:12].values)
       print(song_info)
       df_song_info = pd.DataFrame([song_info])

       loaded_model = pickle.load(open('decision_tree_model.sav', 'rb'))
       prediction = loaded_model.predict(df_song_info)

       song_type = 'Rock' if (prediction == 1) else 'Hip-Hop'

       return title + " song is of type: " + song_type

    return render_template("form.html") 

In [10]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://10de829a64d7.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/Mar/2021 22:16:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2021 22:16:23] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
127.0.0.1 - - [18/Mar/2021 22:19:32] "POST / HTTP/1.1" 200 -


[320000, 0, 211, 3, 1379, 1025, 0, 2017, 2017, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0.99, 0.499, 0.05, 0.94, 0.09, 0.06, 119.0, 0.02, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
